In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
# imports
import sys
sys.path.append("..")
import os
import shutil
from utils.model import Net_mask

import torch
from torch.distributions.multivariate_normal import MultivariateNormal

import matplotlib.pyplot as plt

In [7]:
# seeds
# 1 Traing data 10^4
# 2 Training data 10^5
# 3 Test data 10^3 x 6
# x 10-15, 20-25, ..., 60-65
# 4 RML noise to 10^4 obses
# 5 RML noise to 10^5 obses
# 6 Model weight init

In [8]:
# Common for all data generation
n_param = 10
data_var = 1
data_mean = torch.zeros(n_param)
data_covariance = torch.eye(n_param)*data_var

In [9]:
# Generate trainin data 10^4
torch.manual_seed(1)
n_data = int(1e4)
tensor_data = torch.zeros(n_data, 2*n_param)
tensor_d_sample =  MultivariateNormal(loc=data_mean, covariance_matrix=data_covariance).sample(sample_shape=(n_data,))
tensor_n_masked = torch.randint(1, n_param+1, (n_data,))
tensor_masks = torch.rand(n_data, n_param).argsort(dim=1)
tensor_masks = (tensor_masks < tensor_n_masked.unsqueeze(1))*1
tensor_data[:,:n_param] = tensor_d_sample*tensor_masks
tensor_data[:,n_param:] = tensor_masks
torch.save(tensor_data, f'../data/@/1.0e+04/data/data.pt')
torch.save(tensor_d_sample, f'../data/@/1.0e+04/data/data_obses/data_obses.pt')
torch.save(tensor_masks, f'../data/@/1.0e+04/data/data_masks/data_masks.pt')

In [10]:
# generate trainig data 10^5
torch.manual_seed(2)
n_data = int(1e5)

tensor_data = torch.zeros(n_data, 2*n_param)
tensor_d_sample =  MultivariateNormal(loc=data_mean, covariance_matrix=data_covariance).sample(sample_shape=(n_data,))
tensor_n_masked = torch.randint(1, n_param+1, (n_data,))
tensor_masks = torch.rand(n_data, n_param).argsort(dim=1)
tensor_masks = (tensor_masks < tensor_n_masked.unsqueeze(1))*1
tensor_data[:,:n_param] = tensor_d_sample*tensor_masks
tensor_data[:,n_param:] = tensor_masks
torch.save(tensor_data, f'../data/@/1.0e+05/data/data.pt')
torch.save(tensor_d_sample, f'../data/@/1.0e+05/data/data_obses/data_obses.pt')
torch.save(tensor_masks, f'../data/@/1.0e+05/data/data_masks/data_masks.pt')

In [11]:
# generate test set 6x1000 obses
torch.manual_seed(3)

n_data = int(1e3)

n_max_points = 6
test_data = torch.zeros(n_max_points, n_data, 2*n_param)
for i in range(n_max_points):
    tensor_data = torch.zeros(n_data, 2*n_param)
    tensor_d_sample =  MultivariateNormal(loc=data_mean, covariance_matrix=data_covariance).sample(sample_shape=(n_data,))
    tensor_masks = torch.rand(n_data, n_param).argsort(dim=1)
    tensor_masks = (tensor_masks < i+1)*1
    tensor_data[:,:n_param] = tensor_d_sample*tensor_masks
    tensor_data[:,n_param:] = tensor_masks
    test_data[i] = tensor_data
torch.save(test_data, f'../data/@/test_data/1.0e+03/test_data.pt')

In [12]:
# manual seeds 10-15, 20-25, ..., 60-65

n_data_points = 6
n_points_max = 6
all_points = torch.zeros(100, 2*n_param)
n_data = 1

for i in range(n_points_max):
    for j in range(n_data_points):
        torch.manual_seed(f'{i+1}{j}') # seeds 10-15, 20-25, ..., 60-65
        tensor_data = torch.zeros(n_data, 2*n_param)
        tensor_d_sample =  MultivariateNormal(loc=data_mean, covariance_matrix=data_covariance).sample(sample_shape=(n_data,))
        tensor_n_masked = torch.Tensor([i+1])
        tensor_masks = torch.rand(n_data, n_param).argsort(dim=1)
        tensor_masks = (tensor_masks < tensor_n_masked)*1
        tensor_data[:,:n_param] = tensor_d_sample*tensor_masks
        tensor_data[:,n_param:] = tensor_masks
        all_points[int(f'{i+1}{j}'),:] = tensor_data
torch.save(all_points, f'../data/@/test_data/test_observations/all_test_observations.pt')

In [13]:
tensor_D = torch.tensor(
    [[1, -1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, -1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0,  1, -1, 0, 0, 0, 0, 0, 0],
    [0, 0,  0, 1, -1, 0, 0, 0, 0, 0],
    [0, 0,  0, 0, 1, -1, 0, 0, 0, 0],
    [0, 0,  0, 0, 0, 1, -1, 0, 0, 0],
    [0, 0,  0, 0, 0, 0, 1, -1, 0, 0],
    [0, 0,  0, 0, 0, 0, 0, 1, -1, 0],
    [0, 0,  0, 0, 0, 0, 0, 0, 1, -1],
    [-1, 0, 0, 0, 0, 0, 0, 0, 0, 1],], dtype=torch.float
)
tensor_Q_m = torch.mm(torch.t(tensor_D), tensor_D) 
n_param = tensor_D.size(dim=0)
vu = 10000
inversion_scaling = 1/vu
tensor_Q_m_modified = tensor_Q_m + torch.ones(n_param)*inversion_scaling 
tensor_Sigma_m = torch.inverse(tensor_Q_m_modified)

In [14]:
# RML noise 10^4
torch.manual_seed(4)

sigma2 = 0.1**2 #=0.01
tau2 = 0.3**2 #=0.09
model = Net_mask()
n_weights = sum(p.numel() for p in model.parameters())
n_data = int(1e4)
n_bnn = 100

for i in range(n_bnn):
    path = f'../data/@/1.0e+04/rml_noise/{i}/'
    #os.makedirs(path)
    noise_data = MultivariateNormal(loc=torch.zeros(n_param), covariance_matrix=(torch.eye(n_param)*sigma2)).sample(sample_shape=(n_data,))
    noise_data_regularization =  MultivariateNormal(loc=torch.zeros(n_param), covariance_matrix=(tensor_Sigma_m)).sample(sample_shape=(n_data,))
    noise_parameter = torch.t(MultivariateNormal(loc=torch.zeros(1), covariance_matrix=(torch.eye(1)*tau2)).sample(sample_shape=(n_weights,)))
    torch.save(noise_data, f'{path}noise_data.pt')
    torch.save(noise_data_regularization, f'{path}noise_data_regularization.pt')
    torch.save(noise_parameter, f'{path}noise_parameter.pt')

In [15]:
# RML noise 10^5
torch.manual_seed(5)

sigma2 = 0.1**2 #=0.01
tau2 = 0.3**2 #=0.09
model = Net_mask()
n_weights = sum(p.numel() for p in model.parameters())
n_data = int(1e5)
n_bnn = 100

for i in range(n_bnn):
    path = f'../data/@/1.0e+05/rml_noise/{i}/'
    #os.makedirs(path)
    noise_data = MultivariateNormal(loc=torch.zeros(n_param), covariance_matrix=(torch.eye(n_param)*sigma2)).sample(sample_shape=(n_data,))
    noise_data_regularization =  MultivariateNormal(loc=torch.zeros(n_param), covariance_matrix=(tensor_Sigma_m)).sample(sample_shape=(n_data,))
    noise_parameter = torch.t(MultivariateNormal(loc=torch.zeros(1), covariance_matrix=(torch.eye(1)*tau2)).sample(sample_shape=(n_weights,)))
    torch.save(noise_data, f'{path}noise_data.pt')
    torch.save(noise_data_regularization, f'{path}noise_data_regularization.pt')
    torch.save(noise_parameter, f'{path}noise_parameter.pt')

In [22]:
# Weight initialization
torch.manual_seed(6)
from utils.model import Net_mask

for i in range(100):
    model = Net_mask()
    torch.save(model.state_dict(), f'../data/@/weight_init/rml_e4_{i}.pth')
    print(i,4,  torch.cat([param.view(-1) for param in model.parameters()]))
    model = Net_mask()
    torch.save(model.state_dict(), f'../data/@/weight_init/rml_e5_{i}.pth')
    print(i, 5, torch.cat([param.view(-1) for param in model.parameters()]))
for name in ['map_e4', 'map_e5', 'ml_inf', 'map_inf']:
    model = Net_mask()
    torch.save(model.state_dict(), f'../data/@/weight_init/{name}.pth')


0 4 tensor([ 0.0323,  0.0241,  0.2177,  ...,  0.0099,  0.0236, -0.0335],
       grad_fn=<CatBackward0>)
0 5 tensor([ 0.2141,  0.1882,  0.1617,  ...,  0.0353, -0.0058,  0.0218],
       grad_fn=<CatBackward0>)
1 4 tensor([-0.0680, -0.0767, -0.0836,  ..., -0.0407, -0.0860, -0.0447],
       grad_fn=<CatBackward0>)
1 5 tensor([ 0.1897, -0.0912, -0.0494,  ...,  0.0740, -0.0974,  0.0411],
       grad_fn=<CatBackward0>)
2 4 tensor([ 0.0881,  0.0508,  0.2020,  ..., -0.0592, -0.0814,  0.0940],
       grad_fn=<CatBackward0>)
2 5 tensor([ 0.1747, -0.0097,  0.0909,  ...,  0.0033, -0.0746, -0.0822],
       grad_fn=<CatBackward0>)
3 4 tensor([-0.1864, -0.0050,  0.1208,  ...,  0.0183,  0.0404,  0.0637],
       grad_fn=<CatBackward0>)
3 5 tensor([ 0.2040, -0.0791,  0.1865,  ..., -0.0200,  0.0892, -0.0683],
       grad_fn=<CatBackward0>)
4 4 tensor([ 0.0671,  0.0268, -0.0706,  ..., -0.0347, -0.0815, -0.0888],
       grad_fn=<CatBackward0>)
4 5 tensor([ 0.1180,  0.1843, -0.1520,  ...,  0.0593, -0.0984, -